In [ ]:
#Importar las librerías para acceder a Drive
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
import warnings
import os

#Importar las librerías básicas
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz

from json import dumps
from httplib2 import Http

#Algunas configuraciones
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

warnings.filterwarnings('ignore')

In [ ]:
def cargar_desde_sheets(id: int, hoja: int):
  '''
  Carga la información de un Google Sheets y una hoja específica a un dataframe
  Args:
    - id      (str):          id de la hoja de Sheets
    - hoja    (str):          nombre de la pestaña
  Returns:
    - df      (pd.dataframe): dataframe con la información de Sheets
  '''

  ss = gc.open_by_key(id)
  ss = ss.worksheet(hoja).get_all_values()
  df = pd.DataFrame(ss[1:], columns=ss[0])

  return df

def cargar_a_sheets(df, id, hoja, rango="A1", reemplazo=True):
  '''
  Cargar la información de un dataframe a un Google Sheets en una hoja específica.
  Por defecto, el rango se pegará en el rango A1
  Args:
    - id      (str)           id de la hoja en que se cargará el dataframe
    - df      (pd.DataFrame)  dataframe que desea ser cargado en sheets
    - hoja    (str)           hoja en que se cargará el dataframe
    - rango   (str)           rango en que se cargará el dataframe. Debe ser en notación A1
  '''
  df = df.astype(str)
  workbook = gc.open_by_key(id)
  if reemplazo == True:
    workbook.worksheet(hoja).clear()
  hoja_rango = '{}!{}'.format(hoja, rango)
  workbook.values_update(
      hoja_rango,
      params={
          'valueInputOption': 'USER_ENTERED'
      },
      body={
          'values':[df.columns.values.tolist()] + df.values.tolist()
      }
  )

def cargar_a_bigquery(project_id, dataset_id, table_name, df):
  """
  Tomar un dataframe y cargarlo a una tabla en BigQuery.
  Si la tabla ya existe, la reemplaza
  Args:
  - project_id  (str):
  - dataset_id  (str):
  - table_name  (str):
  - df          (pd.DataFrame):
  """

  client = bigquery.Client(project=project_id)

  table_ref = client.dataset(dataset_id).table(table_name)
  try:
    client.get_table(table_ref)
    client.delete_table(table_ref)
    print(f"Table {table_name} already exists")
  except:
    pass

  table = bigquery.Table(table_ref)
  table = client.create_table(table)

  job_config = bigquery.LoadJobConfig()
  job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
  print(job.result())

def cargar_desde_bigquery(query, project_id):
  """
  Crear una consulta en BigQuery y almacenar los resultados en un dataframe
  Args:
  - query       (str): consulta
  - project_id  (str): id del proyecto en bigquery
  Return:
  - df          (pd.DataFrame): dataframe con la información del query
  """
  client = bigquery.Client(project=project_id)
  query_job = client.query(query)
  df = query_job.to_dataframe()
  return df


from googleapiclient.discovery import build
drive_service = build("drive", "v3", credentials=creds)

def crear_sheets(id_carpeta, nombre_archivo=None, nombres_hojas=None):
  """
  Crear un archivo de Google Sheets en una carpeta específica.
  Al nuevo archivo se le actualizarán los nombres de las hojas.
  Args:
  - id_carpeta (str):     ID de la carpeta en Google Drive.
  - nombre_archivo (str): Nombre del archivo.
  - nombres_hojas (list): Nombres de las hojas.
  Returns:
  - id_archivo (str):     ID del archivo en Google Drive.
  """

  #Crear la metadata del archivo
  metadata = {
      "name": nombre_archivo,
      "mimeType": "application/vnd.google-apps.spreadsheet",
      "parents": [id_carpeta]
  }

  #Crear el archivo y obtener su id
  archivo = drive_service.files().create(body=metadata, fields="id").execute()
  id_archivo = archivo.get("id")

  #Actualizar los nombres de las hojas
  if nombres_hojas:
    archivo = gc.open_by_key(id_archivo)
    for i in range(len(nombres_hojas)):
      if i == 0:
        hoja = archivo.get_worksheet(0)
        hoja.update_title(nombres_hojas[i])
      else:
        archivo.add_worksheet(title=nombres_hojas[i], rows="1000", cols="26")

  return id_archivo